# Tools with MCP ⏰

모델 컨텍스트 프로토콜(MCP)은 AI 에이전트를 외부 도구 및 데이터 소스에 연결하는 표준화된 방법을 제공합니다. `langchain-mcp-adapters`를 사용하여 MCP 서버에 연결해 보겠습니다.

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv("../.env")

# Check and print results
doublecheck_env("../.env")

OPENAI_API_KEY=****VLwA
UPSTAGE_API_KEY=****d51g
LANGCHAIN_TRACING_V2=false
LANGCHAIN_ENDPOINT=****.com
LANGCHAIN_PROJECT=****demy
LANGCHAIN_API_KEY=****fc10
PINECONE_API_KEY=****4e73
HUGGINGFACE_API_KEY=****aRMB
TAVILY_API_KEY=****2UUC
SERPER_API_KEY=****e4cd
WOLFRAM_ALPHA_APPID=****LT74
POLYGON_API_KEY=****8Gso


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply() # 주피터에서 비동기 실행

# 시간대 인식 작업을 위해 mcp-time 서버에 연결
# 이 Go 기반 서버는 현재 시간, 상대 시간 파싱,
# 시간대 변환, 기간 산술, 시간 비교를 위한 도구를 제공
mcp_client = MultiServerMCPClient(# 여러 mcp를 동시에 연결
    {
        "time": { # mcp 서버 이름 (식별자)
            "transport": "stdio", # stdio 표준 입출력을 통해 서버와 통신
            "command": "npx", # node.js 패키지 실행도구
            "args": ["-y", "@theo.foobar/mcp-time"],
            # -y : 자동으로 패키지 설치 확인
            # @theo.foobar/mcp-time: 시간 관련 도구를 제공하는 MCP 서버 패키지
        }
    },
)

# Load tools from the MCP server, 도구 로드
mcp_tools = await mcp_client.get_tools() 
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


- 5가지 도구가 로드 되었다.(서버당 여러개 도구가 로드될 수 있다.)

Create an agent with the MCP-provided time tools.

In [3]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model="openai:gpt-5",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [4]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "현재 샌프란시스코 시간은?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

현재 샌프란시스코 시간은?
================================== Ai Message ==================================
Tool Calls:
  current_time (call_BgDF5fCISPNLF95fjO4MhxOK)
 Call ID: call_BgDF5fCISPNLF95fjO4MhxOK
  Args:
    format: 2006-01-02 15:04:05 MST
    timezone: America/Los_Angeles
================================= Tool Message =================================
Name: current_time

[{'type': 'text', 'text': '2025-12-28 18:47:14 PST', 'id': 'lc_f836495e-dd10-4bbe-8d04-81d7f02b9243'}]
================================== Ai Message ==================================

샌프란시스코 현재 시간: 2025-12-28 18:47:14 PST


- current_time 도구가 호출됨